# Limpieza datos educativos

In [1]:
import pandas as pd
import os
import re

In [2]:
# Cargar el CSV
df = pd.read_csv('republica.csv')
print(f"Cantidad de registros: {df.size}")

Cantidad de registros: 112319


# Filtro
Enfocado en los valores de establecimiento

In [3]:
# Función para limpiar nombres
def normalize_name(name):
    if pd.isnull(name):
        return None  # Cambia "" por None para que se pueda filtrar luego
    name = str(name).upper()
    name = re.sub(r'[\s\-]+', ' ', name)  # reemplaza múltiples espacios o guiones por un solo espacio
    name = name.strip()
    name = re.sub(r'[^\x00-\x7F]+', '', name)  # elimina caracteres no ASCII (invisibles)
    name = name.replace('"', ' ').strip()
    name = name.replace('  ', ' ').strip().replace(' -', '-').replace('- ', '-')
    return name

# Aplicar limpieza
df["ESTABLECIMIENTO_CLEAN"] = df["ESTABLECIMIENTO"].apply(normalize_name)
df["ESTABLECIMIENTO_VARIANTES"] = df["ESTABLECIMIENTO"]

# Eliminar filas donde el nombre limpio es None
df = df[df["ESTABLECIMIENTO_CLEAN"].notna()]

# Agrupar por nombre limpio y juntar resto en sets
def group_sets(col):
    return list(set(col.dropna()))

grouped1 = df.groupby("ESTABLECIMIENTO_CLEAN").agg(group_sets).reset_index()
grouped1 = grouped1.drop(columns=["ESTABLECIMIENTO"])

# Guardar resultado
grouped1.to_csv('limpio.csv', index=False)

In [4]:
print(f"Cantidad de registros: {grouped1.size}")

Cantidad de registros: 63558


## Filtro 2
Revisión de caracteres extra

In [5]:
def group_strings(col):
    flattened = []
    for item in col.dropna():
        # Si el item es string que parece una lista, intenta parsearlo
        if isinstance(item, str) and item.startswith("[") and item.endswith("]"):
            try:
                import ast
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    flattened.extend(parsed)
                else:
                    flattened.append(str(parsed))
            except:
                flattened.append(str(item))
        # Si ya es lista, extiende
        elif isinstance(item, list):
            flattened.extend(item)
        else:
            flattened.append(str(item))

    # Convertir todo a string y eliminar duplicados
    return ", ".join(sorted(set(str(i).strip() for i in flattened)))

In [6]:
from rapidfuzz import fuzz, process

df = pd.read_csv('limpio.csv')

# Agrupación fuzzy
unique_names = df["ESTABLECIMIENTO_CLEAN"].unique()
name_map = {}
visited = set()

for name in unique_names:
    if name in visited:
        continue
    # Buscar similares (umbral de similitud >= 90)
    matches = process.extract(name, unique_names, scorer=fuzz.token_sort_ratio, limit=None)
    group = [match for match, score, _ in matches if score >= 90]
    for g in group:
        name_map[g] = name  # agrupar bajo el nombre base
        visited.add(g)

# Reemplazar por nombre base en el DataFrame
df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO_CLEAN"].map(name_map)

grouped = df.groupby("ESTABLECIMIENTO").agg(group_strings).reset_index()

# Eliminar columnas que ya no necesitas
grouped = grouped.drop(columns=["ESTABLECIMIENTO_CLEAN"])

# Guardar
grouped.to_csv("establecimientos_fuzzy.csv", index=False)


In [7]:
print(f"Cantidad de registros: {grouped.size}")

Cantidad de registros: 51408
